## 5 Blast

O Biopython também tem um pacote específico para fazer comparação de sequências e aqui vamos dar uma olhada no programa Blast. Para rodar Blast Web pela API do NCBI devemos importar o NCBIWWW do pacote Blast.

In [1]:
from Bio.Blast import NCBIWWW
help(NCBIWWW)

Help on module Bio.Blast.NCBIWWW in Bio.Blast:

NAME
    Bio.Blast.NCBIWWW - Code to invoke the NCBI BLAST server over the internet.

DESCRIPTION
    This module provides code to work with the WWW version of BLAST
    provided by the NCBI. https://blast.ncbi.nlm.nih.gov/

FUNCTIONS
    qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_h

No exemplo abaixo fazer rodar o blastn no banco de dados de nucleotídeos e vamos usar como query a sequência do NCBI do o [GI|8332116](https://www.ncbi.nlm.nih.gov/nuccore/237757283). Em seguida vamos salvar esse resultado em um arquivo no formato XML. Salvar o resultado sempre é uma boa prática, pois o Blast pode demorar muito para rodar e você não quer arriscar perder esses resultados. 

In [ ]:
result_handle = NCBIWWW.qblast(program="blastn", database="nt", sequence='237757283', entrez_query="human[Organism]", ncbi_gi=True)

In [ ]:
with open("my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())
result_handle.close()

Agora podemos abrir o resultado do blast com a classe SearchIO e dar uma olhada nos resultados. Esta classe foi especialmente desenvolvida para fazer o parser da saída de diversos programas de alinhamento como Blat, Exonarate e HMMer. O modelo dela consiste uma hierarquia de objetos do Python.

- QueryResult, este é o primeiro nível que consiste em uma única sequência que foi a query;
- Hit, cada query pode ter zero ou multiplos hits, sendo que cada hit representa um único hit no banco de dados;
- HSP (abreviação para high-scoring pair), eles representam regiões com alinhamento significante entre a query e a sequência do bando de dados. Cada hit pode ter 1 ou mais HSPs;
- HSPFragment, representam alinhamentos continuos entre a query e a sequência do banco de dados. Nos resultados do blast os HSPFragments são unificados, então cada HSP só vai ter um HSPFragment. Contudo, outras ferramentas como Exonerate e BLAT produzem HSP com multiplos HSPFragment.

In [ ]:
#listar as queries
from Bio import SearchIO
query = SearchIO.read("my_blast.xml", "blast-xml")

print("%s" % query)

In [ ]:
#listar os hits
for hit in query:
    print("%s\n" % hit)

In [ ]:
#listar os hsp
for hit in query:
    for hsp in hit:
        print("%s\n" % hsp)

In [ ]:
#número de hits do resultado 
print("Número de Hits = %s\n%s" % (len(query), query))

In [ ]:
#Acessar o primeiro hit do resultado
print("%s" % (query[0]))

In [ ]:
#acessar o último resultado
print("%s" % (query[-1]))

In [ ]:
#Slice dos resultados
blast_slice = query[:3]
print("%s" % (blast_slice))

Assim como em um dicionário em Python você pode consultar acessa-lo diretamente através do ID do Hit.

In [ ]:
query["gi|237681126|ref|NR_027676.1|"]

Da mesma foram também é possível consultar se determinado Hit existe.

In [ ]:
"gi|237681126|ref|NR_027676.1|" in query

In [ ]:
"gi|237681126|ref|NR_027676|" in blast_qresult

Também é possívem retornar a posição em que determinado Hit se encontra.

In [ ]:
query.index("gi|237681126|ref|NR_027676.1|")

### 5.1 Atributos dos Hits e HSP
Vamos ver com listar alguns atributos dos Hits e HSPs

In [ ]:
#Aqui pegamos um dos Hits da query
blast_hit = query[45]
blast_hit

In [ ]:
#atributos do Hit
print("Accession: %s" % blast_hit.accession)
print("Blast_id: %s" % blast_hit.blast_id)
print("Description: %s" % blast_hit.description)

In [ ]:
#Aqui pegamos um dos HSPs
blast_hsp = blast_hit.hsps[0]
blast_hsp

In [ ]:
blast_hsp.evalue

In [ ]:
blast_hsp.hit_start

In [ ]:
blast_hsp.query_span

In [ ]:
blast_hsp.aln_span

In [ ]:
blast_hsp.gap_num

In [ ]:
blast_hsp.ident_num

In [ ]:
blast_hsp.query

In [ ]:
blast_hsp.hit

### Exercício 3

Dada um arquivo com uma sequência no formato FASTA que já se encontra no diretório atual com o nome 'execi.fa':
- Abra-o, faça um blastn contra no banco de dados nt; 
- Pegue o primeiro Hit, liste os HSPs e ache uma inserção na query que não tem no Hit do banco de dados;
- Com as coordenadas da inserção selecione apenas essa região na sua query e salve em um arquivo com o nome "dna_message.dna". 
- Após salvar crie uma nova célula e execute com os seguintes comandos e desvende a menssagem:

%%bash

python2.7 dna.py -d dna_message.dna

cat dna_message.decoded